In [ ]:
%load_ext autoreload

In [ ]:
from pandas import read_csv
import math
import keras
from keras.models import Sequential, Model, model_from_json
from keras.layers import concatenate, merge, Dense, LSTM, Input, Reshape, Convolution2D, Deconvolution2D, Flatten, Dropout, MaxPooling2D, Activation
from keras.activations import relu, softmax, linear
from keras.layers.advanced_activations import PReLU, ELU
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
import math
# import cv2
import numpy as np
import csv
from sklearn.utils import shuffle
from pylab import *
from matplotlib import gridspec
cdict = {'red': ((0.0, 1.0, 1.0),
                 (0.125, 1.0, 1.0),
                 (0.25, 1.0, 1.0),
                 (0.5625, 1.0, 1.0),
                 (1.0, 0.0, 0.0)),
         'green': ((0.0, 0.0, 0.0),
                   (0.25, 0.0, 0.0),
                   (0.5625, 1.0, 1.0),
                   (1.0, 1.0, 1.0)),
         'blue': ((0.0, 0.0, 0.0),
                  (0.5, 0.0, 0.0),
                  (1.0, 0.0, 0.0))}
my_cmap = matplotlib.colors.LinearSegmentedColormap('my_colormap',cdict,256)

%matplotlib inline
%autoreload 2
#datapath = '/home/microway/Shuo/CarND/CarND-BehaviorCloning-Project/data-given/'


In [ ]:
import pickle
(Traffic,Speed,data) = pickle.load( open( "speed_short_term.p", "rb" ) )
Count = Traffic[:,:,:,1]
Occup = Traffic[:,:,:,2]
print(Traffic.shape)
print(Speed.shape)
data.tail(5)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler
from lib.Function_DOW_Normalizer import get_certain_dayofweek,MinMax_Normalization,transfer_scale
from lib.Function_CreateInput import shapeback,create_dataset,create_dataset_historyAsFeature,create_dataset_historyAsSecondInput
from lib.Function_VisualizeResult import history_plot,history_plot_historyAsFeature,history_plot_historyAsSecondInput

In [ ]:
epochs = 1000

In [ ]:
train_speed, _, _, _, _ = get_certain_dayofweek(data,Speed[:334],dayofweek = 0)
test_speed = train_speed[-1:]
train_speed = train_speed[:-1]

print('train_speed.shape = ',train_speed.shape)
print('test_speed.shape = ',test_speed.shape)
look_back = 15
mode = 'uni'
train_speed_x,train_speed_y = create_dataset(train_speed,train_speed, look_back, mode)
test_speed_x,test_speed_y = create_dataset(test_speed,test_speed, look_back, mode)
print('look_back = ',look_back)
print('mode = ',mode)
print('train_speed_x.shape = ',train_speed_x.shape)
print('train_speed_y.shape = ',train_speed_y.shape)
print('test_speed_x.shape = ',test_speed_x.shape)
print('test_speed_y.shape = ',test_speed_y.shape)

### Experiment1: input: univariate speed; output: univariate speed; lookback = 15; same day of week

In [ ]:
from lib.Experiment1 import Experiment1

In [ ]:
history = Experiment1(epochs)

In [ ]:
history_plot(history,'images/history_exp1.png','images/test_exp1.png')

### Experiment1.1: input: univariate speed; output: univariate speed; lookback = 15; all year

In [ ]:
from lib.Experiment1_1 import Experiment1_1

In [ ]:
history = Experiment1_1(epochs)

In [ ]:
history_plot(history,'images/history_exp11.png','images/test_exp11.png')

### Experiment2: input: univariate speed; output: univariate speed; lookback = 15; lookback weeks = 15 (as feature); consecutive previous days

In [ ]:
from lib.Experiment2 import Experiment2

In [ ]:
history = Experiment2(epochs)

In [ ]:
history_plot_historyAsFeature(history,'images/history_exp2.png','images/test_exp2.png')

### Experiment2.1: input: univariate speed; output: univariate speed; lookback = 15; lookback weeks = 15 (as feature); previous same day of week

In [ ]:
from lib.Experiment2_1 import Experiment2_1

In [ ]:
history = Experiment2_1(epochs)

In [ ]:
history_plot_historyAsFeature(history,'images/history_exp21.png','images/test_exp21.png')

In [ ]:
# # test_speed = Speed[333:334,:,:]
# train_speed = Speed[:334,:,:]
# dayofweek = data['dayofweek'][333]
# train_speed = Speed[data.index[data['dayofweek'] == dayofweek],:,:]
# train_speed = train_speed[0:48,:,:]
# print('train_speed.shape = ',train_speed.shape)
# # print('test_speed.shape = ',test_speed.shape)

In [ ]:
# look_back = 15
# mode = 'uni'
# train_speed_x,train_speed_y = create_dataset_historyAsFeature(train_speed,train_speed, look_back, mode)
# # test_speed_x,test_speed_y = create_dataset_historyAsFeature(test_speed, look_back, mode)
# test_speed_x = train_speed_x[-1:,:,:,:]
# test_speed_y = train_speed_y[-1:,:,:]
# train_speed_x = train_speed_x[:-1,:,:,:]
# train_speed_y = train_speed_y[:-1,:,:]
# print('look_back = ',look_back)
# print('mode = ',mode)
# print('train_speed_x.shape = ',train_speed_x.shape)
# print('train_speed_y.shape = ',train_speed_y.shape)
# print('test_speed_x.shape = ',test_speed_x.shape)
# print('test_speed_y.shape = ',test_speed_y.shape)

In [ ]:
# plt.plot(test_speed_y[0,390:510,:])  #12-19-2016 Monday 6:30AM - 8:30AM

In [ ]:
# batch_size = train_speed_x.shape[1]

# model = Sequential()
# model.add(LSTM(32, input_shape=(look_back, train_speed_x.shape[3]), stateful=False, return_sequences=True))
# # model.add(Dropout(0.3))
# model.add(LSTM(32, input_shape=(look_back, train_speed_x.shape[3]), stateful=False))
# # model.add(Dropout(0.3))
# model.add(Dense(1))
# model.compile(loss='mean_squared_error', optimizer='adam')

# train_x = np.reshape(train_speed_x,(train_speed_x.shape[0]*train_speed_x.shape[1],train_speed_x.shape[2],train_speed_x.shape[3]))
# train_y = np.reshape(train_speed_y,(train_speed_y.shape[0]*train_speed_y.shape[1],train_speed_y.shape[2]))
# history = model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True)
# # history_plot_historyAsFeature(history)

In [ ]:
# history_plot_historyAsFeature(history,'images/history_exp21.png','images/test_exp21.png')

### Experiment3: input: univariate speed; output: univariate speed; lookback = 15; lookback weeks = 6 (parallel structure); Monday only for training, Monday for testing


In [ ]:
# train_speed, _, _, _, _ = get_certain_dayofweek(Speed[:334],dayofweek = 0)
# # test_speed = train_speed[-1:]
# # train_speed = train_speed[:-1]

# print('train_speed.shape = ',train_speed.shape)
# # print('test_speed.shape = ',test_speed.shape)

In [ ]:
# look_back = 15
# look_back_days = 6
# mode = 'uni'
# train_speed_x1,train_speed_x2,train_speed_y = create_dataset_historyAsSecondInput(train_speed,train_speed, look_back, look_back_days, mode)

# test_speed_x1 = train_speed_x1[-1:,:,:,:]
# test_speed_x2 = train_speed_x2[-1:,:,:,:]
# test_speed_y = train_speed_y[-1:,:,:]
# train_speed_x1 = train_speed_x1[:-1,:,:,:]
# train_speed_x2 = train_speed_x2[:-1,:,:,:]
# train_speed_y = train_speed_y[:-1,:,:]

# print('look_back = ',look_back)
# print('look_back_days = ',look_back_days)
# print('mode = ',mode)
# print('train_speed_x1.shape = ',train_speed_x1.shape)
# print('train_speed_x2.shape = ',train_speed_x2.shape)
# print('train_speed_y.shape = ',train_speed_y.shape)
# print('test_speed_x1.shape = ',test_speed_x1.shape)
# print('test_speed_x2.shape = ',test_speed_x2.shape)
# print('test_speed_y.shape = ',test_speed_y.shape)

In [ ]:
# plt.plot(test_speed_y[0,390:510,:])  #12-19-2016 Monday 6:30AM - 8:30AM

In [ ]:
# batch_size = train_speed_x.shape[1]


# todaySequence = Input(shape=(look_back, train_speed_x1.shape[3]),name='todaySequence')
# h1=LSTM(32, input_shape=(look_back, train_speed_x1.shape[3]), stateful=False, return_sequences=True)(todaySequence)
# h1=LSTM(32, input_shape=(look_back, train_speed_x1.shape[3]), stateful=False)(h1)

# historySequence = Input(shape=(look_back_days, train_speed_x2.shape[3]),name='historySequence')
# h2=LSTM(32, input_shape=(look_back, train_speed_x2.shape[3]), stateful=False, return_sequences=True)(historySequence)
# h2=LSTM(32, input_shape=(look_back, train_speed_x2.shape[3]), stateful=False)(h2)

# h3 = keras.layers.concatenate([h1, h2])
# predictedSpeed = Dense(1,name='predictedSpeed')(h3)

# model = Model(inputs=[todaySequence, historySequence], outputs=[predictedSpeed])

# model.compile(loss='mean_squared_error', optimizer='adam')

# # model.compile(optimizer='rmsprop',
# #               loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
# #               loss_weights={'main_output': 1., 'aux_output': 0.2})

# train_x1 = np.reshape(train_speed_x1,(train_speed_x1.shape[0]*train_speed_x1.shape[1],train_speed_x1.shape[2],train_speed_x1.shape[3]))
# train_x2 = np.reshape(train_speed_x2,(train_speed_x2.shape[0]*train_speed_x2.shape[1],train_speed_x2.shape[2],train_speed_x2.shape[3]))
# train_y = np.reshape(train_speed_y,(train_speed_y.shape[0]*train_speed_y.shape[1],train_speed_y.shape[2]))

# history = model.fit({'todaySequence': train_x1, 'historySequence': train_x2},
#           {'predictedSpeed': train_y},
#           epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True)

# # history_plot_historyAsSecondInput(history)

In [ ]:
# history_plot_historyAsSecondInput(history,'images/history_exp3.png','images/test_exp3.png')

### Experiment3.1: input: univariate speed; output: univariate speed; lookback = 15; lookback weeks = 6 (parallel structure); all weekdays for training, monday for testing


In [ ]:
# # test_speed = Speed[333:334,:,:]
# # train_speed = Speed[:334,:,:]
# # dayofweek = data['dayofweek'][333]
# train_speed0,_,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 0)
# train_speed1,_,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 1)
# train_speed2,_,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 2)
# train_speed3,_,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 3)
# train_speed4,_,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 4)
# train_speed5,_,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 5)
# train_speed6,_,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 6)
# # train_speed0 = train_speed0[0:48,:,:]
# print('train_speed0.shape = ',train_speed.shape)
# # print('test_speed.shape = ',test_speed.shape)

In [ ]:
# look_back = 15
# look_back_days = 6
# mode = 'uni'
# train_speed_x1,train_speed_x2,train_speed_y = create_dataset_historyAsSecondInput(train_speed0,train_speed0, look_back, look_back_days, mode)

# test_speed_x1 = train_speed_x1[-1:,:,:,:]
# test_speed_x2 = train_speed_x2[-1:,:,:,:]
# test_speed_y = train_speed_y[-1:,:,:]
# train_speed_x1 = train_speed_x1[:-1,:,:,:]
# train_speed_x2 = train_speed_x2[:-1,:,:,:]
# train_speed_y = train_speed_y[:-1,:,:]


# train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed1,train_speed1, look_back, look_back_days, mode)
# train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
# train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
# train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
# train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed2,train_speed2, look_back, look_back_days, mode)
# train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
# train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
# train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
# train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed3,train_speed3, look_back, look_back_days, mode)
# train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
# train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
# train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
# train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed4,train_speed4, look_back, look_back_days, mode)
# train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
# train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
# train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
# train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed5,train_speed5, look_back, look_back_days, mode)
# train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
# train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
# train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
# train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed6,train_speed6, look_back, look_back_days, mode)
# train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
# train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
# train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)

# print('look_back = ',look_back)
# print('look_back_days = ',look_back_days)
# print('mode = ',mode)
# print('train_speed_x1.shape = ',train_speed_x1.shape)
# print('train_speed_x2.shape = ',train_speed_x2.shape)
# print('train_speed_y.shape = ',train_speed_y.shape)
# print('test_speed_x1.shape = ',test_speed_x1.shape)
# print('test_speed_x2.shape = ',test_speed_x2.shape)
# print('test_speed_y.shape = ',test_speed_y.shape)

In [ ]:
# plt.plot(test_speed_y[0,390:510,:])  #12-19-2016 Monday 6:30AM - 8:30AM

In [ ]:
# batch_size = train_speed_x.shape[1]

# todaySequence = Input(shape=(look_back, train_speed_x1.shape[3]),name='todaySequence')
# h1=LSTM(32, input_shape=(look_back, train_speed_x1.shape[3]), stateful=False, return_sequences=True)(todaySequence)
# h1=LSTM(32, input_shape=(look_back, train_speed_x1.shape[3]), stateful=False)(h1)

# historySequence = Input(shape=(look_back_days, train_speed_x2.shape[3]),name='historySequence')
# h2=LSTM(32, input_shape=(look_back, train_speed_x2.shape[3]), stateful=False, return_sequences=True)(historySequence)
# h2=LSTM(32, input_shape=(look_back, train_speed_x2.shape[3]), stateful=False)(h2)

# h3 = keras.layers.concatenate([h1, h2])
# predictedSpeed = Dense(1,name='predictedSpeed')(h3)

# model = Model(inputs=[todaySequence, historySequence], outputs=[predictedSpeed])

# model.compile(loss='mean_squared_error', optimizer='adam')

# # model.compile(optimizer='rmsprop',
# #               loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
# #               loss_weights={'main_output': 1., 'aux_output': 0.2})

# train_x1 = np.reshape(train_speed_x1,(train_speed_x1.shape[0]*train_speed_x1.shape[1],train_speed_x1.shape[2],train_speed_x1.shape[3]))
# train_x2 = np.reshape(train_speed_x2,(train_speed_x2.shape[0]*train_speed_x2.shape[1],train_speed_x2.shape[2],train_speed_x2.shape[3]))
# train_y = np.reshape(train_speed_y,(train_speed_y.shape[0]*train_speed_y.shape[1],train_speed_y.shape[2]))

# history = model.fit({'todaySequence': train_x1, 'historySequence': train_x2},
#           {'predictedSpeed': train_y},
#           epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True)

# # history_plot_historyAsSecondInput(history)

In [ ]:
# history_plot_historyAsSecondInput(history,'images/history_exp31.png','images/test_exp31.png')

### Experiment4: input: univariate speed; output: univariate delta speed; lookback = 15

In [ ]:
# # Mon, Mon_delta, Mon_Z, mon_mean, mon_std = get_certain_dayofweek(Speed,dayofweek = 0)
# train_speed, train_speed_y, _, mean0, _ = get_certain_dayofweek(Speed[:334],dayofweek = 0)
# test_speed = train_speed[-1:]
# train_speed = train_speed[:-1]
# test_speed_y = train_speed_y[-1:]
# train_speed_y = train_speed_y[:-1]


# print('train_speed.shape = ',train_speed.shape)
# print('test_speed.shape = ',test_speed.shape)

# test_speed = Speed[333:334,:,:]
# train_speed = Speed[:334,:,:]
train_speed0,train_speed_y0,_,mean0,_ = get_certain_dayofweek(Speed[:334],dayofweek = 0)
train_speed1,train_speed_y1,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 1)
train_speed2,train_speed_y2,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 2)
train_speed3,train_speed_y3,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 3)
train_speed4,train_speed_y4,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 4)
train_speed5,train_speed_y5,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 5)
train_speed6,train_speed_y6,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 6)
# train_speed0 = train_speed0[0:48,:,:]
print('train_speed0.shape = ',train_speed.shape)
# print('test_speed.shape = ',test_speed.shape)

In [ ]:
# look_back = 15
# mode = 'uni'
# train_speed_x,train_speed_y = create_dataset(train_speed,train_speed_y, look_back, mode)
# test_speed_x,test_speed_y = create_dataset(test_speed,test_speed_y, look_back, mode)
# print('look_back = ',look_back)
# print('mode = ',mode)
# print('train_speed_x.shape = ',train_speed_x.shape)
# print('train_speed_y.shape = ',train_speed_y.shape)
# print('test_speed_x.shape = ',test_speed_x.shape)
# print('test_speed_y.shape = ',test_speed_y.shape)

look_back = 15
mode = 'uni'
train_speed_x,train_speed_y = create_dataset(train_speed0,train_speed_y0, look_back, mode)

test_speed_x = train_speed_x[-1:,:,:,:]
test_speed_y = train_speed_y[-1:,:,:]
train_speed_x = train_speed_x[:-1,:,:,:]
train_speed_y = train_speed_y[:-1,:,:]

train_speed_x10,train_speed_y0 = create_dataset(train_speed1,train_speed_y1, look_back,mode)
train_speed_x = np.concatenate((train_speed_x,train_speed_x10),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_y0 = create_dataset(train_speed2,train_speed_y2, look_back,mode)
train_speed_x = np.concatenate((train_speed_x,train_speed_x10),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_y0 = create_dataset(train_speed3,train_speed_y3, look_back,mode)
train_speed_x = np.concatenate((train_speed_x,train_speed_x10),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_y0 = create_dataset(train_speed4,train_speed_y4, look_back,mode)
train_speed_x = np.concatenate((train_speed_x,train_speed_x10),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_y0 = create_dataset(train_speed5,train_speed_y5, look_back,mode)
train_speed_x = np.concatenate((train_speed_x,train_speed_x10),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_y0 = create_dataset(train_speed6,train_speed_y6, look_back,mode)
train_speed_x = np.concatenate((train_speed_x,train_speed_x10),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)

print('look_back = ',look_back)
print('look_back_days = ',look_back_days)
print('mode = ',mode)
print('train_speed_x.shape = ',train_speed_x.shape)
print('train_speed_y.shape = ',train_speed_y.shape)
print('test_speed_x.shape = ',test_speed_x.shape)
print('test_speed_y.shape = ',test_speed_y.shape)

In [ ]:
# plt.plot(test_speed_y[0,390:510,:]+mean0[390:510,0:1])  #12-19-2016 Monday 6:30AM - 8:30AM

In [ ]:
batch_size = train_speed_x.shape[1]

model = Sequential()
model.add(LSTM(32, input_shape=(look_back, train_speed_x.shape[3]), stateful=False, return_sequences=True))
# model.add(Dropout(0.3))
model.add(LSTM(32, input_shape=(look_back, train_speed_x.shape[3]), stateful=False))
# model.add(Dropout(0.3))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

train_x = np.reshape(train_speed_x,(train_speed_x.shape[0]*train_speed_x.shape[1],train_speed_x.shape[2],train_speed_x.shape[3]))
train_y = np.reshape(train_speed_y,(train_speed_y.shape[0]*train_speed_y.shape[1],train_speed_y.shape[2]))
history = model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True)
# history_plot(history,mean0)

In [ ]:
history_plot(history,'images/history_exp4.png','images/test_exp4.png',a=mean0,b=mean0)
history_plot(history,'images/history_exp4_delta.png','images/test_exp4_delta.png',b=mean0)

### Experiment5: input: univariate speed; output: univariate delta speed; lookback = 15; lookback weeks = 15 (as feature);

### Experiment6: input: univariate speed; output: univariate delta speed; lookback = 15; lookback weeks = 6 (parallel structure)

In [ ]:
# test_speed = Speed[333:334,:,:]
# train_speed = Speed[:334,:,:]
# dayofweek = data['dayofweek'][333]
train_speed0,train_speed_y0,_,mean0,_ = get_certain_dayofweek(Speed[:334],dayofweek = 0)
train_speed1,train_speed_y1,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 1)
train_speed2,train_speed_y2,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 2)
train_speed3,train_speed_y3,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 3)
train_speed4,train_speed_y4,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 4)
train_speed5,train_speed_y5,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 5)
train_speed6,train_speed_y6,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 6)
# train_speed0 = train_speed0[0:48,:,:]
print('train_speed0.shape = ',train_speed.shape)
# print('test_speed.shape = ',test_speed.shape)

In [ ]:
look_back = 15
look_back_days = 6
mode = 'uni'
train_speed_x1,train_speed_x2,train_speed_y = create_dataset_historyAsSecondInput(train_speed0,train_speed_y0, look_back, look_back_days, mode)

test_speed_x1 = train_speed_x1[-1:,:,:,:]
test_speed_x2 = train_speed_x2[-1:,:,:,:]
test_speed_y = train_speed_y[-1:,:,:]
train_speed_x1 = train_speed_x1[:-1,:,:,:]
train_speed_x2 = train_speed_x2[:-1,:,:,:]
train_speed_y = train_speed_y[:-1,:,:]


train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed1,train_speed_y1, look_back, look_back_days, mode)
train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed2,train_speed_y2, look_back, look_back_days, mode)
train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed3,train_speed_y3, look_back, look_back_days, mode)
train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed4,train_speed_y4, look_back, look_back_days, mode)
train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed5,train_speed_y5, look_back, look_back_days, mode)
train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed6,train_speed_y6, look_back, look_back_days, mode)
train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)

print('look_back = ',look_back)
print('look_back_days = ',look_back_days)
print('mode = ',mode)
print('train_speed_x1.shape = ',train_speed_x1.shape)
print('train_speed_x2.shape = ',train_speed_x2.shape)
print('train_speed_y.shape = ',train_speed_y.shape)
print('test_speed_x1.shape = ',test_speed_x1.shape)
print('test_speed_x2.shape = ',test_speed_x2.shape)
print('test_speed_y.shape = ',test_speed_y.shape)

In [ ]:
# plt.plot(test_speed_y[0,390:510,:]+mean0[390:510,0:1])  #12-19-2016 Monday 6:30AM - 8:30AM

In [ ]:

batch_size = train_speed_x.shape[1]

todaySequence = Input(shape=(look_back, train_speed_x1.shape[3]),name='todaySequence')
h1=LSTM(32, input_shape=(look_back, train_speed_x1.shape[3]), stateful=False, return_sequences=True)(todaySequence)

h1=LSTM(32, input_shape=(look_back, train_speed_x1.shape[3]), stateful=False,name='h1')(todaySequence)
h1= BatchNormalization()(h1)

historySequence = Input(shape=(look_back_days, train_speed_x2.shape[3]),name='historySequence')
h2=LSTM(32, input_shape=(look_back, train_speed_x2.shape[3]), stateful=False, return_sequences=True)(historySequence)
h2=LSTM(32, input_shape=(look_back, train_speed_x2.shape[3]), stateful=False,name='h2')(h2)

h3 = keras.layers.concatenate([h1, h2],name='h3')
# h3 = keras.layers.concatenate([h1, h2],name='h3')

predictedSpeed = Dense(1,name='predictedSpeed')(h3)

model = Model(inputs=[todaySequence, historySequence], outputs=[predictedSpeed])

model.compile(loss='mean_squared_error', optimizer='adam')

# model.compile(optimizer='rmsprop',
#               loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
#               loss_weights={'main_output': 1., 'aux_output': 0.2})

train_x1 = np.reshape(train_speed_x1,(train_speed_x1.shape[0]*train_speed_x1.shape[1],train_speed_x1.shape[2],train_speed_x1.shape[3]))
train_x2 = np.reshape(train_speed_x2,(train_speed_x2.shape[0]*train_speed_x2.shape[1],train_speed_x2.shape[2],train_speed_x2.shape[3]))
train_y = np.reshape(train_speed_y,(train_speed_y.shape[0]*train_speed_y.shape[1],train_speed_y.shape[2]))

history = model.fit({'todaySequence': train_x1, 'historySequence': train_x2},
          {'predictedSpeed': train_y},
          epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True)

# history_plot_historyAsSecondInput(history,mean0)

In [ ]:
# intermediate_layer_model_h1 = Model(inputs=model.input,
#                                  outputs=model.get_layer('h1').output)
# intermediate_output_h1 = intermediate_layer_model_h1.predict([train_x1[:1],train_x2[:1]])
# intermediate_layer_model_h2 = Model(inputs=model.input,
#                                  outputs=model.get_layer('h2').output)
# intermediate_output_h2 = intermediate_layer_model_h2.predict([train_x1[:1],train_x2[:1]])
# intermediate_layer_model_h3 = Model(inputs=model.input,
#                                  outputs=model.get_layer('h3').output)
# intermediate_output_h3 = intermediate_layer_model_h3.predict([train_x1[:1],train_x2[:1]])

In [ ]:
history_plot_historyAsSecondInput(history,'images/history_exp6.png','images/test_exp6.png',a=mean0,b=mean0)
history_plot_historyAsSecondInput(history,'images/history_exp6_delta.png','images/test_exp6_delta.png',b=mean0)

In [ ]:
def history_plot_multi(history_object,image1='image1',image2='image2',image3='image3',a=np.zeros((test_speed_y.shape[1],1)),b=np.zeros((test_speed_y.shape[1],1))):
    
    trainScore = [];
    for i in range(len(train_speed_x)):
        trainScore.append(model.evaluate(train_speed_x[i,:,:,:], train_speed_y[i,:,:], batch_size=batch_size, verbose=0))
    trainScore = np.mean(trainScore)
    print('Train Score: ', trainScore)
    testScore = [];
    for i in range(len(test_speed_x)):
        testScore.append(model.evaluate(test_speed_x[i,:,:,:], test_speed_y[i,:,:], batch_size=batch_size, verbose=0))
    testScore = np.mean(testScore)
    print('Test Score: ', testScore)

    fig2 = plt.figure(figsize=(15,5))
    plt.plot(history_object.history['loss'])
    plt.title('model mean squared error loss (Train Score:' + str(trainScore) + ' Test Score:' + str(testScore))
    plt.ylabel('mean squared error loss')
    plt.xlabel('epoch')
    fig2.savefig(image1, bbox_inches='tight')
    
    look_ahead = 120
    start = 390
    trainPredict = test_speed_x[0,start,:,:]
    predictions = np.zeros((look_ahead,15))

    for i in range(look_ahead):
        prediction = model.predict(np.array([trainPredict]), batch_size=batch_size)
        predictions[i] = prediction
        trainPredict = np.vstack([trainPredict[1:],prediction+b[(start+i):(start+i+1),:1]])

    fig1 = plt.figure(figsize=(12,10))
    ax1 = plt.subplot(2,1,1)
    ax1.set_title('prediction at the start of day', fontsize=20)
    
    plt.plot(np.arange(look_ahead),predictions[:,:1]+a[start:(start+look_ahead),:1],'r',label="prediction")
    plt.plot(np.arange(look_ahead),test_speed_y[0,start:(start+look_ahead),:1]+a[start:(start+look_ahead),:1],label="test function")
    plt.legend()

    predictions_1 = np.zeros((look_ahead,15))

    for i in range(look_ahead):
        trainPredict = test_speed_x[0,start+i,:,:]
        prediction = model.predict(np.array([trainPredict]), batch_size=batch_size)
        predictions_1[i] = prediction


    ax2 = plt.subplot(2,1,2)
    ax2.set_title('prediction using real-time data', fontsize=20)
    plt.plot(np.arange(look_ahead),predictions_1[:,:1]+a[start:(start+look_ahead),:1],'r',label="prediction")
    plt.plot(np.arange(look_ahead),test_speed_y[0,start:(start+look_ahead),:1]+a[start:(start+look_ahead),:1],label="test function")
    plt.legend()

    fig1.savefig(image2, bbox_inches='tight')

    fig3 = plt.figure(figsize=(12,10))
    ax1 = plt.subplot(3,1,1)
    ax1.set_title('test data', fontsize=20)
    plt.pcolor(test_speed_y[0,start:(start+look_ahead),:].transpose()+a[start:(start+look_ahead),:].transpose(),cmap=my_cmap, vmin=20, vmax=70)
    
    ax2 = plt.subplot(3,1,2)
    ax2.set_title('prediction at the start', fontsize=20)
    plt.pcolor(predictions.transpose()+a[start:(start+look_ahead),:].transpose(),cmap=my_cmap, vmin=20, vmax=70)
    
    ax3 = plt.subplot(3,1,3)
    ax3.set_title('prediction using real-time data', fontsize=20)
    plt.pcolor(predictions_1.transpose()+a[start:(start+look_ahead),:].transpose(),cmap=my_cmap, vmin=20, vmax=70)
    
    fig3.savefig(image3, bbox_inches='tight')

def history_plot_multi_historyAsSecondInput(history_object,image1='image1',image2='image2',image3='image3',a=np.zeros((test_speed_y.shape[1],1)),b=np.zeros((test_speed_y.shape[1],1))):
  
    trainScore = [];
    for i in range(len(train_speed_x1)):
        trainScore.append(model.evaluate([train_speed_x1[i,:,:,:],train_speed_x2[i,:,:,:]], train_speed_y[i,:,:], batch_size=batch_size, verbose=0))
    trainScore = np.mean(trainScore)
    print('Train Score: ', trainScore)
    testScore = [];
    for i in range(len(test_speed_x1)):
        testScore.append(model.evaluate([test_speed_x1[i,:,:,:],test_speed_x2[i,:,:,:]], test_speed_y[i,:,:], batch_size=batch_size, verbose=0))
    testScore = np.mean(testScore)
    print('Test Score: ', testScore)

    fig2 = plt.figure(figsize=(15,5))
    plt.plot(history_object.history['loss'])
    plt.title('model mean squared error loss (Train Score:' + str(trainScore) + ' Test Score:' + str(testScore))
    plt.ylabel('mean squared error loss')
    plt.xlabel('epoch')
    fig2.savefig(image1, bbox_inches='tight')
    
    look_ahead = 120
    start = 390
    trainPredict = test_speed_x1[0,start,:,:]  
    predictions = np.zeros((look_ahead,15))

    for i in range(look_ahead):
        input2 = test_speed_x2[0,start+i,:,:]
        prediction = model.predict([np.array([trainPredict]),np.array([input2])], batch_size=batch_size)
        predictions[i] = prediction
        trainPredict = np.vstack([trainPredict[1:],prediction+b[(start+i):(start+i+1),:1]])

    fig1 = plt.figure(figsize=(12,10))
    ax1 = plt.subplot(2,1,1)
    ax1.set_title('prediction at the start of day', fontsize=20)
    plt.plot(np.arange(look_ahead),predictions[:,:1]+a[start:(start+look_ahead),:1],'r',label="prediction")
    plt.plot(np.arange(look_ahead),test_speed_y[0,start:(start+look_ahead),:1]+a[start:(start+look_ahead),:1],label="test function")
    plt.legend()

    predictions_1 = np.zeros((look_ahead,15))

    for i in range(look_ahead):
        trainPredict = test_speed_x1[0,start+i,:,:]
        input2 = test_speed_x2[0,start+i,:,:]
        prediction = model.predict([np.array([trainPredict]),np.array([input2])], batch_size=batch_size)
        predictions_1[i] = prediction


    ax2 = plt.subplot(2,1,2)
    ax2.set_title('prediction using real-time data', fontsize=20)
    plt.plot(np.arange(look_ahead),predictions_1[:,:1]+a[start:(start+look_ahead),:1],'r',label="prediction")
    plt.plot(np.arange(look_ahead),test_speed_y[0,start:(start+look_ahead),:1]+a[start:(start+look_ahead),:1],label="test function")
    plt.legend()

    fig1.savefig(image2, bbox_inches='tight')

    fig3 = plt.figure(figsize=(12,10))
    ax1 = plt.subplot(3,1,1)
    ax1.set_title('test data', fontsize=20)
    plt.pcolor(test_speed_y[0,start:(start+look_ahead),:].transpose()+a[start:(start+look_ahead),:].transpose(),cmap=my_cmap, vmin=20, vmax=70)
    
    ax2 = plt.subplot(3,1,2)
    ax2.set_title('prediction at the start', fontsize=20)
    plt.pcolor(predictions.transpose()+a[start:(start+look_ahead),:].transpose(),cmap=my_cmap, vmin=20, vmax=70)
    
    ax3 = plt.subplot(3,1,3)
    ax3.set_title('prediction using real-time data', fontsize=20)
    plt.pcolor(predictions_1.transpose()+a[start:(start+look_ahead),:].transpose(),cmap=my_cmap, vmin=20, vmax=70)
    
    fig3.savefig(image3, bbox_inches='tight')


### Experiment7: input: multivariate speed; output: multivariate speed; lookback = 15

In [ ]:
test_speed = Speed[333:334,:,:]
train_speed = Speed[:333,:,:]
print('train_speed.shape = ',train_speed.shape)
print('test_speed.shape = ',test_speed.shape)

In [ ]:
look_back = 15
mode = 'multi'
train_speed_x,train_speed_y = create_dataset(train_speed,train_speed, look_back, mode)
test_speed_x,test_speed_y = create_dataset(test_speed,test_speed, look_back, mode)
print('look_back = ',look_back)
print('mode = ',mode)
print('train_speed_x.shape = ',train_speed_x.shape)
print('train_speed_y.shape = ',train_speed_y.shape)
print('test_speed_x.shape = ',test_speed_x.shape)
print('test_speed_y.shape = ',test_speed_y.shape)

In [ ]:
# %matplotlib inline
# plt.plot(test_speed_y[0,390:510,:1])  #12-19-2016 Monday 6:30AM - 8:30AM
# plt.pcolor(test_speed_y[0,390:510,:].transpose(),cmap=my_cmap, vmin=20, vmax=70)  #12-19-2016 Monday 6:30AM - 8:30AM

In [ ]:
batch_size = train_speed_x.shape[1]

model = Sequential()
model.add(LSTM(32, input_shape=(look_back, train_speed_x.shape[3]), stateful=False, return_sequences=True))
# model.add(Dropout(0.3))
model.add(LSTM(32, input_shape=(look_back, train_speed_x.shape[3]), stateful=False))
# model.add(Dropout(0.3))
# model.add(Flatten())
model.add(Dense(train_speed_y.shape[2]))
model.compile(loss='mean_squared_error', optimizer='adam')

train_x = np.reshape(train_speed_x,(train_speed_x.shape[0]*train_speed_x.shape[1],train_speed_x.shape[2],train_speed_x.shape[3]))
train_y = np.reshape(train_speed_y,(train_speed_y.shape[0]*train_speed_y.shape[1],train_speed_y.shape[2]))
history = model.fit(train_x, train_y,epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True)
# history_plot(history)

In [ ]:
history_plot_multi(history,'images/history_exp7.png','images/test_exp7.png','images/heatmap_exp7.png')

### Experiment8: input: multivariate speed; output: multivariate speed; lookback = 15; lookback weeks = 15 (as feature);

### Experiment9: input: multivariate speed; output: multivariate speed; lookback = 15; lookback weeks = 6 (parallel structure)

In [ ]:
# test_speed = Speed[333:334,:,:]
# train_speed = Speed[:334,:,:]
# dayofweek = data['dayofweek'][333]
train_speed0,_,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 0)
train_speed1,_,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 1)
train_speed2,_,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 2)
train_speed3,_,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 3)
train_speed4,_,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 4)
train_speed5,_,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 5)
train_speed6,_,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 6)
# train_speed0 = train_speed0[0:48,:,:]
print('train_speed0.shape = ',train_speed.shape)
# print('test_speed.shape = ',test_speed.shape)

In [ ]:
look_back = 15
look_back_days = 6
mode = 'multi'
train_speed_x1,train_speed_x2,train_speed_y = create_dataset_historyAsSecondInput(train_speed0,train_speed0, look_back, look_back_days, mode)

test_speed_x1 = train_speed_x1[-1:,:,:,:]
test_speed_x2 = train_speed_x2[-1:,:,:,:]
test_speed_y = train_speed_y[-1:,:,:]
train_speed_x1 = train_speed_x1[:-1,:,:,:]
train_speed_x2 = train_speed_x2[:-1,:,:,:]
train_speed_y = train_speed_y[:-1,:,:]


train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed1,train_speed1, look_back, look_back_days, mode)
train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed2,train_speed2, look_back, look_back_days, mode)
train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed3,train_speed3, look_back, look_back_days, mode)
train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed4,train_speed4, look_back, look_back_days, mode)
train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed5,train_speed5, look_back, look_back_days, mode)
train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed6,train_speed6, look_back, look_back_days, mode)
train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)

print('look_back = ',look_back)
print('look_back_days = ',look_back_days)
print('mode = ',mode)
print('train_speed_x1.shape = ',train_speed_x1.shape)
print('train_speed_x2.shape = ',train_speed_x2.shape)
print('train_speed_y.shape = ',train_speed_y.shape)
print('test_speed_x1.shape = ',test_speed_x1.shape)
print('test_speed_x2.shape = ',test_speed_x2.shape)
print('test_speed_y.shape = ',test_speed_y.shape)

In [ ]:
# plt.plot(test_speed_y[0,390:510,:1])  #12-19-2016 Monday 6:30AM - 8:30AM
# plt.pcolor(test_speed_y[0,390:510,:].transpose(),cmap=my_cmap, vmin=20, vmax=70)  #12-19-2016 Monday 6:30AM - 8:30AM

In [ ]:
batch_size = train_speed_x.shape[1]

todaySequence = Input(shape=(look_back, train_speed_x1.shape[3]),name='todaySequence')
h1=LSTM(32, input_shape=(look_back, train_speed_x1.shape[3]), stateful=False, return_sequences=True)(todaySequence)
h1=LSTM(32, input_shape=(look_back, train_speed_x1.shape[3]), stateful=False)(h1)

historySequence = Input(shape=(look_back_days, train_speed_x2.shape[3]),name='historySequence')
h2=LSTM(32, input_shape=(look_back, train_speed_x2.shape[3]), stateful=False, return_sequences=True)(historySequence)
h2=LSTM(32, input_shape=(look_back, train_speed_x2.shape[3]), stateful=False)(h2)

h3 = keras.layers.concatenate([h1, h2])
predictedSpeed = Dense(train_speed_y.shape[2],name='predictedSpeed')(h3)

model = Model(inputs=[todaySequence, historySequence], outputs=[predictedSpeed])

model.compile(loss='mean_squared_error', optimizer='adam')

# model.compile(optimizer='rmsprop',
#               loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
#               loss_weights={'main_output': 1., 'aux_output': 0.2})

train_x1 = np.reshape(train_speed_x1,(train_speed_x1.shape[0]*train_speed_x1.shape[1],train_speed_x1.shape[2],train_speed_x1.shape[3]))
train_x2 = np.reshape(train_speed_x2,(train_speed_x2.shape[0]*train_speed_x2.shape[1],train_speed_x2.shape[2],train_speed_x2.shape[3]))
train_y = np.reshape(train_speed_y,(train_speed_y.shape[0]*train_speed_y.shape[1],train_speed_y.shape[2]))

history = model.fit({'todaySequence': train_x1, 'historySequence': train_x2},
          {'predictedSpeed': train_y},
          epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True)

# history_plot_historyAsSecondInput(history)

In [ ]:
history_plot_multi_historyAsSecondInput(history,'images/history_exp9.png','images/test_exp9.png','images/heatmap_exp9.png')

### Experiment10: input: multivariate speed; output: multivariate delta speed; lookback = 15

In [ ]:
# # Mon, Mon_delta, Mon_Z, mon_mean, mon_std = get_certain_dayofweek(Speed,dayofweek = 0)
# train_speed, train_speed_y, _, mean0, _ = get_certain_dayofweek(Speed[:334],dayofweek = 0)
# test_speed = train_speed[-1:]
# train_speed = train_speed[:-1]
# test_speed_y = train_speed_y[-1:]
# train_speed_y = train_speed_y[:-1]


# print('train_speed.shape = ',train_speed.shape)
# print('test_speed.shape = ',test_speed.shape)

# test_speed = Speed[333:334,:,:]
# train_speed = Speed[:334,:,:]
train_speed0,train_speed_y0,_,mean0,_ = get_certain_dayofweek(Speed[:334],dayofweek = 0)
train_speed1,train_speed_y1,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 1)
train_speed2,train_speed_y2,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 2)
train_speed3,train_speed_y3,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 3)
train_speed4,train_speed_y4,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 4)
train_speed5,train_speed_y5,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 5)
train_speed6,train_speed_y6,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 6)
# train_speed0 = train_speed0[0:48,:,:]
print('train_speed0.shape = ',train_speed.shape)
# print('test_speed.shape = ',test_speed.shape)

In [ ]:
# look_back = 15
# mode = 'uni'
# train_speed_x,train_speed_y = create_dataset(train_speed,train_speed_y, look_back, mode)
# test_speed_x,test_speed_y = create_dataset(test_speed,test_speed_y, look_back, mode)
# print('look_back = ',look_back)
# print('mode = ',mode)
# print('train_speed_x.shape = ',train_speed_x.shape)
# print('train_speed_y.shape = ',train_speed_y.shape)
# print('test_speed_x.shape = ',test_speed_x.shape)
# print('test_speed_y.shape = ',test_speed_y.shape)

look_back = 15
mode = 'multi'
train_speed_x,train_speed_y = create_dataset(train_speed0,train_speed_y0, look_back, mode)

test_speed_x = train_speed_x[-1:,:,:,:]
test_speed_y = train_speed_y[-1:,:,:]
train_speed_x = train_speed_x[:-1,:,:,:]
train_speed_y = train_speed_y[:-1,:,:]

train_speed_x10,train_speed_y0 = create_dataset(train_speed1,train_speed_y1, look_back,mode)
train_speed_x = np.concatenate((train_speed_x,train_speed_x10),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_y0 = create_dataset(train_speed2,train_speed_y2, look_back,mode)
train_speed_x = np.concatenate((train_speed_x,train_speed_x10),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_y0 = create_dataset(train_speed3,train_speed_y3, look_back,mode)
train_speed_x = np.concatenate((train_speed_x,train_speed_x10),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_y0 = create_dataset(train_speed4,train_speed_y4, look_back,mode)
train_speed_x = np.concatenate((train_speed_x,train_speed_x10),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_y0 = create_dataset(train_speed5,train_speed_y5, look_back,mode)
train_speed_x = np.concatenate((train_speed_x,train_speed_x10),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_y0 = create_dataset(train_speed6,train_speed_y6, look_back,mode)
train_speed_x = np.concatenate((train_speed_x,train_speed_x10),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)

print('look_back = ',look_back)
print('look_back_days = ',look_back_days)
print('mode = ',mode)
print('train_speed_x.shape = ',train_speed_x.shape)
print('train_speed_y.shape = ',train_speed_y.shape)
print('test_speed_x.shape = ',test_speed_x.shape)
print('test_speed_y.shape = ',test_speed_y.shape)

In [ ]:
# plt.plot(test_speed_y[0,390:510,:1]+mean0[390:510,:1])  #12-19-2016 Monday 6:30AM - 8:30AM
# plt.pcolor(mean0[390:510,:].transpose(),cmap=my_cmap, vmin=20, vmax=70)  #12-19-2016 Monday 6:30AM - 8:30AM
# plt.pcolor(test_speed_y[0,390:510,:].transpose()+mean0[390:510,:].transpose(),cmap=my_cmap, vmin=20, vmax=70)  #12-19-2016 Monday 6:30AM - 8:30AM

In [ ]:
batch_size = train_speed_x.shape[1]

model = Sequential()
model.add(LSTM(32, input_shape=(look_back, train_speed_x.shape[3]), stateful=False, return_sequences=True))
# model.add(Dropout(0.3))
model.add(LSTM(32, input_shape=(look_back, train_speed_x.shape[3]), stateful=False))
# model.add(Dropout(0.3))
model.add(Dense(train_speed_y.shape[2]))
model.compile(loss='mean_squared_error', optimizer='adam')

train_x = np.reshape(train_speed_x,(train_speed_x.shape[0]*train_speed_x.shape[1],train_speed_x.shape[2],train_speed_x.shape[3]))
train_y = np.reshape(train_speed_y,(train_speed_y.shape[0]*train_speed_y.shape[1],train_speed_y.shape[2]))
history = model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True)
# history_plot(history,mean0)

In [ ]:
history_plot_multi(history,'images/history_exp10.png','images/test_exp10.png','images/heatmap_exp10.png',a=mean0,b=mean0)
history_plot_multi(history,'images/history_exp10_delta.png','images/test_exp10_delta.png','images/heatmap_exp10_delta.png',b=mean0)

### Experiment11: input: multivariate speed; output: multivariate delta speed; lookback = 15; lookback weeks = 15 (as feature);

### Experiment12: input: multivariate speed; output: multivariate delta speed; lookback = 15; lookback weeks = 6 (parallel structure)

In [ ]:
# test_speed = Speed[333:334,:,:]
# train_speed = Speed[:334,:,:]
# dayofweek = data['dayofweek'][333]
train_speed0,train_speed_y0,_,mean0,_ = get_certain_dayofweek(Speed[:334],dayofweek = 0)
train_speed1,train_speed_y1,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 1)
train_speed2,train_speed_y2,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 2)
train_speed3,train_speed_y3,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 3)
train_speed4,train_speed_y4,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 4)
train_speed5,train_speed_y5,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 5)
train_speed6,train_speed_y6,_,_,_ = get_certain_dayofweek(Speed[:334],dayofweek = 6)
# train_speed0 = train_speed0[0:48,:,:]
print('train_speed0.shape = ',train_speed.shape)
# print('test_speed.shape = ',test_speed.shape)

In [ ]:
look_back = 15
look_back_days = 6
mode = 'multi'
train_speed_x1,train_speed_x2,train_speed_y = create_dataset_historyAsSecondInput(train_speed0,train_speed_y0, look_back, look_back_days, mode)

test_speed_x1 = train_speed_x1[-1:,:,:,:]
test_speed_x2 = train_speed_x2[-1:,:,:,:]
test_speed_y = train_speed_y[-1:,:,:]
train_speed_x1 = train_speed_x1[:-1,:,:,:]
train_speed_x2 = train_speed_x2[:-1,:,:,:]
train_speed_y = train_speed_y[:-1,:,:]


train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed1,train_speed_y1, look_back, look_back_days, mode)
train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed2,train_speed_y2, look_back, look_back_days, mode)
train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed3,train_speed_y3, look_back, look_back_days, mode)
train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed4,train_speed_y4, look_back, look_back_days, mode)
train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed5,train_speed_y5, look_back, look_back_days, mode)
train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)
train_speed_x10,train_speed_x20,train_speed_y0 = create_dataset_historyAsSecondInput(train_speed6,train_speed_y6, look_back, look_back_days, mode)
train_speed_x1 = np.concatenate((train_speed_x1,train_speed_x10),axis=0)
train_speed_x2 = np.concatenate((train_speed_x2,train_speed_x20),axis=0)
train_speed_y = np.concatenate((train_speed_y,train_speed_y0),axis=0)

print('look_back = ',look_back)
print('look_back_days = ',look_back_days)
print('mode = ',mode)
print('train_speed_x1.shape = ',train_speed_x1.shape)
print('train_speed_x2.shape = ',train_speed_x2.shape)
print('train_speed_y.shape = ',train_speed_y.shape)
print('test_speed_x1.shape = ',test_speed_x1.shape)
print('test_speed_x2.shape = ',test_speed_x2.shape)
print('test_speed_y.shape = ',test_speed_y.shape)

In [ ]:
# plt.plot(test_speed_y[0,390:510,:1]+mean0[390:510,:1])  #12-19-2016 Monday 6:30AM - 8:30AM
# plt.pcolor(mean0[390:510,:].transpose(),cmap=my_cmap, vmin=20, vmax=70)  #12-19-2016 Monday 6:30AM - 8:30AM
# plt.pcolor(test_speed_y[0,390:510,:].transpose()+mean0[390:510,:].transpose(),cmap=my_cmap, vmin=20, vmax=70)  #12-19-2016 Monday 6:30AM - 8:30AM

In [ ]:

batch_size = train_speed_x.shape[1]

todaySequence = Input(shape=(look_back, train_speed_x1.shape[3]),name='todaySequence')
h1=LSTM(32, input_shape=(look_back, train_speed_x1.shape[3]), stateful=False, return_sequences=True)(todaySequence)

h1=LSTM(32, input_shape=(look_back, train_speed_x1.shape[3]), stateful=False,name='h1')(todaySequence)

historySequence = Input(shape=(look_back_days, train_speed_x2.shape[3]),name='historySequence')
h2=LSTM(32, input_shape=(look_back, train_speed_x2.shape[3]), stateful=False, return_sequences=True)(historySequence)
h2=LSTM(32, input_shape=(look_back, train_speed_x2.shape[3]), stateful=False,name='h2')(h2)

h3 = keras.layers.concatenate([h1, h2],name='h3')
# h3 = keras.layers.concatenate([h1, h2],name='h3')

predictedSpeed = Dense(train_speed_y.shape[2],name='predictedSpeed')(h3)

model = Model(inputs=[todaySequence, historySequence], outputs=[predictedSpeed])

model.compile(loss='mean_squared_error', optimizer='adam')

# model.compile(optimizer='rmsprop',
#               loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
#               loss_weights={'main_output': 1., 'aux_output': 0.2})

train_x1 = np.reshape(train_speed_x1,(train_speed_x1.shape[0]*train_speed_x1.shape[1],train_speed_x1.shape[2],train_speed_x1.shape[3]))
train_x2 = np.reshape(train_speed_x2,(train_speed_x2.shape[0]*train_speed_x2.shape[1],train_speed_x2.shape[2],train_speed_x2.shape[3]))
train_y = np.reshape(train_speed_y,(train_speed_y.shape[0]*train_speed_y.shape[1],train_speed_y.shape[2]))

history = model.fit({'todaySequence': train_x1, 'historySequence': train_x2},
          {'predictedSpeed': train_y},
          epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True)

# history_plot_historyAsSecondInput(history,mean0)

In [ ]:
history_plot_multi_historyAsSecondInput(history,'images/history_exp12.png','images/test_exp12.png','images/heatmap_exp12.png',a=mean0,b=mean0)
history_plot_multi_historyAsSecondInput(history,'images/history_exp12_delta.png','images/test_exp12_delta.png','images/heatmap_exp12_delta.png',b=mean0)

In [ ]:
def history_plot_multi_co(history_object,image1='image1',image2='image2',image3='image3',a=np.zeros((test_speed_y.shape[1],1)),b=np.zeros((test_speed_y.shape[1],1))):
    
    trainScore = [];
    for i in range(len(train_speed_x)):
        trainScore.append(model.evaluate(train_speed_x[i,:,:,:], train_speed_y[i,:,:], batch_size=batch_size, verbose=0))
    trainScore = np.mean(trainScore)
    print('Train Score: ', trainScore)
    testScore = [];
    for i in range(len(test_speed_x)):
        testScore.append(model.evaluate(test_speed_x[i,:,:,:], test_speed_y[i,:,:], batch_size=batch_size, verbose=0))
    testScore = np.mean(testScore)
    print('Test Score: ', testScore)

    fig2 = plt.figure(figsize=(15,5))
    plt.plot(history_object.history['loss'])
    plt.title('model mean squared error loss (Train Score:' + str(trainScore) + ' Test Score:' + str(testScore))
    plt.ylabel('mean squared error loss')
    plt.xlabel('epoch')
#     fig2.savefig(image1, bbox_inches='tight')
    
    look_ahead = 120
    start = 390
    trainPredict = test_speed_x[0,start,:,:]
    predictions = np.zeros((look_ahead,15))

    for i in range(look_ahead):
        prediction = model.predict(np.array([trainPredict]), batch_size=batch_size)
        predictions[i] = prediction
        trainPredict = np.vstack([trainPredict[1:],prediction+b[(start+i):(start+i+1),:1]])

    fig1 = plt.figure(figsize=(12,10))
    ax1 = plt.subplot(2,1,1)
    ax1.set_title('prediction at the start of day', fontsize=20)
    
    plt.plot(np.arange(look_ahead),predictions[:,:1]+a[start:(start+look_ahead),:1],'r',label="prediction")
    plt.plot(np.arange(look_ahead),test_speed_y[0,start:(start+look_ahead),:1]+a[start:(start+look_ahead),:1],label="test function")
    plt.legend()

    predictions_1 = np.zeros((look_ahead,15))

    for i in range(look_ahead):
        trainPredict = test_speed_x[0,start+i,:,:]
        prediction = model.predict(np.array([trainPredict]), batch_size=batch_size)
        predictions_1[i] = prediction


    ax2 = plt.subplot(2,1,2)
    ax2.set_title('prediction using real-time data', fontsize=20)
    plt.plot(np.arange(look_ahead),predictions_1[:,:1]+a[start:(start+look_ahead),:1],'r',label="prediction")
    plt.plot(np.arange(look_ahead),test_speed_y[0,start:(start+look_ahead),:1]+a[start:(start+look_ahead),:1],label="test function")
    plt.legend()

#     fig1.savefig(image2, bbox_inches='tight')

    fig3 = plt.figure(figsize=(12,10))
    ax1 = plt.subplot(3,1,1)
    ax1.set_title('test data', fontsize=20)
    plt.pcolor(test_speed_y[0,start:(start+look_ahead),:].transpose()+a[start:(start+look_ahead),:].transpose(),cmap=my_cmap, vmin=20, vmax=70)
    
    ax2 = plt.subplot(3,1,2)
    ax2.set_title('prediction at the start', fontsize=20)
    plt.pcolor(predictions.transpose()+a[start:(start+look_ahead),:].transpose(),cmap=my_cmap, vmin=20, vmax=70)
    
    ax3 = plt.subplot(3,1,3)
    ax3.set_title('prediction using real-time data', fontsize=20)
    plt.pcolor(predictions_1.transpose()+a[start:(start+look_ahead),:].transpose(),cmap=my_cmap, vmin=20, vmax=70)
    
#     fig3.savefig(image3, bbox_inches='tight')

def history_plot_multi_co_historyAsSecondInput(history_object,image1='image1',image2='image2',image3='image3',a=np.zeros((test_speed_y.shape[1],1)),b=np.zeros((test_speed_y.shape[1],1))):
  
    trainScore = [];
    for i in range(len(train_speed_x1)):
        trainScore.append(model.evaluate([train_speed_x1[i,:,:,:],train_speed_x2[i,:,:,:]], train_speed_y[i,:,:], batch_size=batch_size, verbose=0))
    trainScore = np.mean(trainScore)
    print('Train Score: ', trainScore)
    testScore = [];
    for i in range(len(test_speed_x1)):
        testScore.append(model.evaluate([test_speed_x1[i,:,:,:],test_speed_x2[i,:,:,:]], test_speed_y[i,:,:], batch_size=batch_size, verbose=0))
    testScore = np.mean(testScore)
    print('Test Score: ', testScore)

    fig2 = plt.figure(figsize=(15,5))
    plt.plot(history_object.history['loss'])
    plt.title('model mean squared error loss (Train Score:' + str(trainScore) + ' Test Score:' + str(testScore))
    plt.ylabel('mean squared error loss')
    plt.xlabel('epoch')
#     fig2.savefig(image1, bbox_inches='tight')
    
    look_ahead = 120
    start = 390
    trainPredict = test_speed_x1[0,start,:,:]  
    predictions = np.zeros((look_ahead,15))

    for i in range(look_ahead):
        input2 = test_speed_x2[0,start+i,:,:]
        prediction = model.predict([np.array([trainPredict]),np.array([input2])], batch_size=batch_size)
        predictions[i] = prediction
        trainPredict = np.vstack([trainPredict[1:],prediction+b[(start+i):(start+i+1),:1]])

    fig1 = plt.figure(figsize=(12,10))
    ax1 = plt.subplot(2,1,1)
    ax1.set_title('prediction at the start of day', fontsize=20)
    plt.plot(np.arange(look_ahead),predictions[:,:1]+a[start:(start+look_ahead),:1],'r',label="prediction")
    plt.plot(np.arange(look_ahead),test_speed_y[0,start:(start+look_ahead),:1]+a[start:(start+look_ahead),:1],label="test function")
    plt.legend()

    predictions_1 = np.zeros((look_ahead,15))

    for i in range(look_ahead):
        trainPredict = test_speed_x1[0,start+i,:,:]
        input2 = test_speed_x2[0,start+i,:,:]
        prediction = model.predict([np.array([trainPredict]),np.array([input2])], batch_size=batch_size)
        predictions_1[i] = prediction


    ax2 = plt.subplot(2,1,2)
    ax2.set_title('prediction using real-time data', fontsize=20)
    plt.plot(np.arange(look_ahead),predictions_1[:,:1]+a[start:(start+look_ahead),:1],'r',label="prediction")
    plt.plot(np.arange(look_ahead),test_speed_y[0,start:(start+look_ahead),:1]+a[start:(start+look_ahead),:1],label="test function")
    plt.legend()

#     fig1.savefig(image2, bbox_inches='tight')

    fig3 = plt.figure(figsize=(12,10))
    ax1 = plt.subplot(3,1,1)
    ax1.set_title('test data', fontsize=20)
    plt.pcolor(test_speed_y[0,start:(start+look_ahead),:].transpose()+a[start:(start+look_ahead),:].transpose(),cmap=my_cmap, vmin=20, vmax=70)
    
    ax2 = plt.subplot(3,1,2)
    ax2.set_title('prediction at the start', fontsize=20)
    plt.pcolor(predictions.transpose()+a[start:(start+look_ahead),:].transpose(),cmap=my_cmap, vmin=20, vmax=70)
    
    ax3 = plt.subplot(3,1,3)
    ax3.set_title('prediction using real-time data', fontsize=20)
    plt.pcolor(predictions_1.transpose()+a[start:(start+look_ahead),:].transpose(),cmap=my_cmap, vmin=20, vmax=70)
    
#     fig3.savefig(image3, bbox_inches='tight')


### Experiment13: input: multivariate speed, count, occup; output: multivariate speed; lookback = 15

In [ ]:
test_speed,_ = MinMax_Normalization(Speed[333:334,:,:])
test_speed_y = Speed[333:334,:,:]
test_count,_ = MinMax_Normalization(Count[333:334,:,:])
test_occup,_ = MinMax_Normalization(Occup[333:334,:,:])
train_speed,scaler = MinMax_Normalization(Speed[:333,:,:])
train_speed_y = Speed[:333,:,:]
train_count,_ = MinMax_Normalization(Count[:333,:,:])
train_occup,_ = MinMax_Normalization(Occup[:333,:,:])
print('train_speed.shape = ',train_speed.shape)
print('test_speed.shape = ',test_speed.shape)

In [ ]:
look_back = 15
mode = 'multi'
train_speed_x,train_speed_y = create_dataset(train_speed,train_speed_y, look_back, mode)
train_speed_c,_ = create_dataset(train_count,train_count, look_back, mode)
train_speed_o,_ = create_dataset(train_occup,train_occup, look_back, mode)
train_speed_x = np.concatenate((train_speed_x,train_speed_c,train_speed_o),axis=-1)

test_speed_x,test_speed_y = create_dataset(test_speed,test_speed_y, look_back, mode)
test_speed_c,_ = create_dataset(test_count,test_count, look_back, mode)
test_speed_o,_ = create_dataset(test_occup,test_occup, look_back, mode)
test_speed_x = np.concatenate((test_speed_x,test_speed_c,test_speed_o),axis=-1)

print('look_back = ',look_back)
print('mode = ',mode)
print('train_speed_x.shape = ',train_speed_x.shape)
print('train_speed_y.shape = ',train_speed_y.shape)
print('test_speed_x.shape = ',test_speed_x.shape)
print('test_speed_y.shape = ',test_speed_y.shape)

In [ ]:
# %matplotlib inline
# plt.plot(test_speed_y[0,390:510,:1])  #12-19-2016 Monday 6:30AM - 8:30AM
# plt.pcolor(test_speed_y[0,390:510,:].transpose(),cmap=my_cmap, vmin=20, vmax=70)  #12-19-2016 Monday 6:30AM - 8:30AM

In [ ]:
batch_size = train_speed_x.shape[1]

model = Sequential()
model.add(LSTM(32, input_shape=(look_back, train_speed_x.shape[3]), stateful=False, return_sequences=True))
# model.add(Dropout(0.3))
model.add(LSTM(32, input_shape=(look_back, train_speed_x.shape[3]), stateful=False))
# model.add(Dropout(0.3))
# model.add(Flatten())
model.add(Dense(train_speed_y.shape[2]))
model.compile(loss='mean_squared_error', optimizer='adam')

train_x = np.reshape(train_speed_x,(train_speed_x.shape[0]*train_speed_x.shape[1],train_speed_x.shape[2],train_speed_x.shape[3]))
train_y = np.reshape(train_speed_y,(train_speed_y.shape[0]*train_speed_y.shape[1],train_speed_y.shape[2]))
history = model.fit(train_x, train_y,epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True)
# history_plot(history)

In [ ]:
    look_ahead = 120
    start = 390
    trainPredict = test_speed_x[0,start,:,:]
    predictions = np.zeros((look_ahead,15))

#     for i in range(look_ahead):
#         prediction = model.predict(np.array([trainPredict]), batch_size=batch_size)
#         predictions[i] = prediction
#         trainPredict = np.vstack([trainPredict[1:],prediction])

In [ ]:
trainPredict.shape

In [ ]:
samples_normalized = scaler.transform(samples)

In [ ]:
# history_plot_multi_co(history)

### Experiment14: input: multivariate speed, count, occup; output: multivariate speed; lookback = 15; lookback weeks = 15 (as feature);

### Experiment15: input: multivariate speed, count, occup; output: multivariate speed; lookback = 15; lookback weeks = 6 (parallel structure)

### Experiment16: input: multivariate speed, count, occup; output: multivariate delta speed; lookback = 15

### Experiment17: input: multivariate speed, count, occup; output: multivariate delta speed; lookback = 15; lookback weeks = 15 (as feature);

### Experiment18: input: multivariate speed, count, occup; output: multivariate delta speed; lookback = 15; lookback weeks = 6 (parallel structure)